# Pelabelan Data Menggunakan Lexicon

In [1]:
import pandas as pd
import re
import string
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_csv("datasetReady.csv")
df

,tweets,label
0,"['tanjung', 'balai', 'laksana', 'vaksinasi']",1.0
1,"['dukung', 'program', 'juta', 'vaksinasi', 'lu...",1.0
2,"['sebut', 'lonjak', 'covid', 'kendali']",1.0
3,"['lampung', 'gelar', 'vaksinasi', 'covid', 'an...",1.0
4,"['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...",1.0
...,...,...
787,"['grab', 'luncur', 'pusat', 'vaksinasi', 'rama...",1.0
788,"['cegah', 'sebar', 'covid', 'sentra', 'vaksina...",1.0
789,"['neng', 'liat', 'animo', 'teman', 'teman', 'v...",1.0
790,"['lonjak', 'covid', 'izin', 'vaksin', 'vaksina...",0.0


In [3]:
word_dict = {}
for i in range(0,len(df['tweets'])):
    sentence = df['tweets'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1

In [4]:
len(word_dict)

2234

In [5]:
len({k:v for (k,v) in word_dict.items() if v < 4})

1803

In [6]:
negasi = ['bukan','tidak','ga','gk']
lexicon = pd.read_csv('lexicon/modified_full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
                               |(lexicon['word'] == 'tidak')
                               |(lexicon['word'] == 'ga')
                               |(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)

In [7]:
len(lexicon)

10248

In [8]:
lexicon.head(10)

,word,weight,number_of_words
0,hai,3,1
1,merekam,2,1
2,ekstensif,3,1
3,paripurna,1,1
4,detail,2,1
5,pernik,3,1
6,belas,2,1
7,welas,4,1
8,kabung,1,1
9,rahayu,4,1


In [9]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

In [10]:
len(lexicon_word)

10248

In [11]:
ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

1570

In [12]:
len({k:v for (k,v) in word_dict.items() if ((k in ns_words)&(v>3)) })

250

In [13]:
ns_words_list = {k:v for (k,v) in word_dict.items() if ((k in ns_words)&(v>3))}

In [14]:
sort_orders = sorted(ns_words_list.items(), key=lambda x: x[1], reverse=True)
sort_orders=sort_orders[0:20]
for i in sort_orders:
    print(i[0], i[1])

' 8347
, 7565
'vaksinasi 863
[ 792
] 792
'covid 258
'vaksin 153
'laksana 114
'tinjau 94
'massal 92
'program 76
'warga 69
'masyarakat 65
'jakarta 64
'bogor 61
'giat 53
'kota 48
'stadion 46
'lansia 43
'usia 36


In [15]:
lexicon['number_of_words'].value_counts()

1    9536
2     686
3      24
4       2
Name: number_of_words, dtype: int64

In [16]:
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['tweets'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [17]:
len(sentiment_list)

792

In [18]:
print(senrow.shape[0])

792


In [19]:
sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)

In [20]:
df_sen.head(10)

,balai,dukung,juta,gelar,anggota,keluarga,rumah,sakit,bandar,kobar,...,pola,tempoh,lebih,bangkit,neng,animo,lupa,bahas,prosedur,sentiment
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-3
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,-3
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-4
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-5
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [21]:
cek_df = pd.DataFrame([])
cek_df['text'] = df['tweets'].copy()
cek_df['sentiment']  = df_sen['sentiment'].copy()

In [22]:
cek_df.head(10)

,text,sentiment
0,"['tanjung', 'balai', 'laksana', 'vaksinasi']",-3
1,"['dukung', 'program', 'juta', 'vaksinasi', 'lu...",8
2,"['sebut', 'lonjak', 'covid', 'kendali']",0
3,"['lampung', 'gelar', 'vaksinasi', 'covid', 'an...",-3
4,"['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...",2
5,"['columbia', 'asia', 'indonesia', 'dukung', 'p...",4
6,"['kam', 'laksana', 'vaksinasi', 'warga', 'usia...",-4
7,"['kalbar', 'serius', 'vaksinasi', 'lansia']",-5
8,"['salah', 'banget', 'vaksinasi', 'uji', 'menan...",-2
9,"['silah', 'simak', 'jadwal', 'vaksinasi', 'tin...",2


In [24]:
cek_df['sentiment'] = df_sen['sentiment'].copy()
df.loc[cek_df['sentiment']>=0, 'sentiment'] = 1 #positive
df.loc[cek_df['sentiment']<0, 'sentiment'] = 0 #negative
# df.loc[cek_df['sentiment']==0, 'sentiment'] = 2 #netral
# df.drop(df[cek_df.sentiment==0].index, inplace=True)
# df.reset_index()
df

,tweets,label,sentiment
0,"['tanjung', 'balai', 'laksana', 'vaksinasi']",1.0,0.0
1,"['dukung', 'program', 'juta', 'vaksinasi', 'lu...",1.0,1.0
2,"['sebut', 'lonjak', 'covid', 'kendali']",1.0,1.0
3,"['lampung', 'gelar', 'vaksinasi', 'covid', 'an...",1.0,0.0
4,"['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...",1.0,1.0
...,...,...,...
787,"['grab', 'luncur', 'pusat', 'vaksinasi', 'rama...",1.0,1.0
788,"['cegah', 'sebar', 'covid', 'sentra', 'vaksina...",1.0,1.0
789,"['neng', 'liat', 'animo', 'teman', 'teman', 'v...",1.0,1.0
790,"['lonjak', 'covid', 'izin', 'vaksin', 'vaksina...",0.0,0.0


In [26]:
df.drop("label",axis=1)

,tweets,sentiment
0,"['tanjung', 'balai', 'laksana', 'vaksinasi']",0.0
1,"['dukung', 'program', 'juta', 'vaksinasi', 'lu...",1.0
2,"['sebut', 'lonjak', 'covid', 'kendali']",1.0
3,"['lampung', 'gelar', 'vaksinasi', 'covid', 'an...",0.0
4,"['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...",1.0
...,...,...
787,"['grab', 'luncur', 'pusat', 'vaksinasi', 'rama...",1.0
788,"['cegah', 'sebar', 'covid', 'sentra', 'vaksina...",1.0
789,"['neng', 'liat', 'animo', 'teman', 'teman', 'v...",1.0
790,"['lonjak', 'covid', 'izin', 'vaksin', 'vaksina...",0.0


In [28]:
df

,tweets,label,sentiment
0,"['tanjung', 'balai', 'laksana', 'vaksinasi']",1.0,0.0
1,"['dukung', 'program', 'juta', 'vaksinasi', 'lu...",1.0,1.0
2,"['sebut', 'lonjak', 'covid', 'kendali']",1.0,1.0
3,"['lampung', 'gelar', 'vaksinasi', 'covid', 'an...",1.0,0.0
4,"['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...",1.0,1.0
...,...,...,...
787,"['grab', 'luncur', 'pusat', 'vaksinasi', 'rama...",1.0,1.0
788,"['cegah', 'sebar', 'covid', 'sentra', 'vaksina...",1.0,1.0
789,"['neng', 'liat', 'animo', 'teman', 'teman', 'v...",1.0,1.0
790,"['lonjak', 'covid', 'izin', 'vaksin', 'vaksina...",0.0,0.0


In [27]:
df.to_csv('lexiconDataLabelling.csv')
# df.to_excel('data fix\datalabeled.xlsx')